<a href="https://colab.research.google.com/github/anshupandey/MSA-analytics/blob/main/Model_Monitoring/Lab6_BeeRelevant_Model_Monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bee-Relevant Hull Insurance Dataset - Model Monitoring & Retraining Labs
This notebook covers:
- Drift detection using PSI, Kolmogorov–Smirnov Test, and Jensen–Shannon Divergence
- Model training, evaluation, and retraining strategy
- Building a monitoring dashboard

Dataset: Bee-Relevant Ocean Hull Insurance (Anonymized)

In [23]:
import pandas as pd

# Load the dataset
df = pd.read_excel("Ocean Hull Data for Bee-Relevant - Anonymised.xlsx")
df.head()

,Dummy_Policy_Reference,Dummy_Vessel_Reference,YearOfAccount,PolicyPremium,ExposureYears,Entity,AggregateDeductible_USD,HM_GrossAchievedPremium_polccy,HM_Coverage,HM_AdditionalMachineryDeductible_USD,...,VesselFlag,TradingZone,TradingArea,VesselClass,VesselPIClub,DeductibleUSDPerSelectedTonnage,IMOFlag,AggregateDeductibleFlag,ClaimCount,ClaimIncurred
0,1781,1,2020,45657.73,1.0,MSAM,NaN,NaN,RISKSEXCCOLLLIABS,NaN,...,Palau,NaN,WorldWide,0,Unknown,35.714286,Yes,NO,0,0.0000
1,1780,1,2021,9833.95,1.0,MSAM,NaN,NaN,RISKSEXCCOLLLIABS,NaN,...,Palau,NaN,WorldWide,0,Unknown,35.714286,Yes,NO,0,0.0000
2,1779,1,2021,7525.32,1.0,MSAM,NaN,NaN,RISKSEXCLIABS,NaN,...,NaN,Local,Local,XX,XX,20.054437,No,NO,0,0.0000
3,1778,1,2021,1256.67,1.0,MSAM,NaN,NaN,ALLRISKS,NaN,...,NaN,Local,Local,XX,XX,NaN,No,NO,1,48430.1296
4,1778,2,2021,1256.67,1.0,MSAM,NaN,NaN,ALLRISKS,NaN,...,NaN,Local,Local,XX,XX,NaN,No,NO,0,0.0000


In [24]:
df.shape

(20422, 32)

## Step 1: Exploratory Data Analysis
Let's explore the data to identify key features and target variables.

In [25]:
# Summary and null counts
df.info()
df.isnull().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20422 entries, 0 to 20421
Data columns (total 32 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Dummy_Policy_Reference                20422 non-null  int64  
 1   Dummy_Vessel_Reference                20422 non-null  int64  
 2   YearOfAccount                         20422 non-null  int64  
 3   PolicyPremium                         20422 non-null  float64
 4   ExposureYears                         20422 non-null  float64
 5   Entity                                20422 non-null  object 
 6   AggregateDeductible_USD               1655 non-null   float64
 7   HM_GrossAchievedPremium_polccy        2058 non-null   float64
 8   HM_Coverage                           20422 non-null  object 
 9   HM_AdditionalMachineryDeductible_USD  644 non-null    float64
 10  HM_Deductible_USD                     20422 non-null  float64
 11  Vessel_EnginePo

,0
HM_AdditionalMachineryDeductible_USD,19778
AggregateDeductible_USD,18767
HM_GrossAchievedPremium_polccy,18364
Tonnage_Type_To_Use,17929
VesselFlag,14300
Deadweight,10326
TradingZone,5057
DeductibleUSDPerSelectedTonnage,5027
GrossTonnage,944
SelectedTonnage,923


## Lab 2: Drift Detection - PSI, KS Test, and Jensen–Shannon Divergence
We'll compare the distribution of selected features between a historical training slice and a recent/current slice.

In [26]:
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats import ks_2samp, entropy
import matplotlib.pyplot as plt


In [27]:
df.Vessel_Age.fillna(df.Vessel_Age.median(),inplace=True)

/tmp/ipython-input-27-3461856681.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Vessel_Age.fillna(df.Vessel_Age.median(),inplace=True)


In [28]:
# Drop rows with missing values
df_clean = df.T.dropna().T
df_clean.shape

(20422, 21)

In [29]:
df_clean['YearOfAccount'].describe()

,YearOfAccount
count,20422
unique,8
top,2018
freq,3708


In [30]:
# Create a binary split assuming 'Policy Inception Date' defines time
df_clean['Policy Inception Date'] = pd.to_datetime(df_clean['YearOfAccount'],format='%Y')
cutoff_date = df_clean['Policy Inception Date'].quantile(0.5)
cutoff_date

Timestamp('2019-01-01 00:00:00')

In [31]:
df_clean['Policy Inception Date'].describe()

,Policy Inception Date
count,20422
mean,2018-11-17 22:43:12.733326848
min,2016-01-01 00:00:00
25%,2017-01-01 00:00:00
50%,2019-01-01 00:00:00
75%,2021-01-01 00:00:00
max,2023-01-01 00:00:00


In [32]:
train_slice = df_clean[df_clean['Policy Inception Date'] <= cutoff_date]
current_slice = df_clean[df_clean['Policy Inception Date'] > cutoff_date]

In [33]:
train_slice.columns

Index(['Dummy_Policy_Reference', 'Dummy_Vessel_Reference', 'YearOfAccount',
       'PolicyPremium', 'ExposureYears', 'Entity', 'HM_Coverage',
       'HM_Deductible_USD', 'Vessel_EnginePower', 'Vessel_Age',
       'Vessel_Currency', 'Vessel_Type', 'Vessel_Type_Group',
       'Ocean_Going_Or_Craft', 'Vessel_Value_USD', 'VesselClass',
       'VesselPIClub', 'IMOFlag', 'AggregateDeductibleFlag', 'ClaimCount',
       'ClaimIncurred', 'Policy Inception Date'],
      dtype='object')

In [34]:
# Pick numeric features for analysis
features = ['PolicyPremium', 'Vessel_Value_USD', 'Vessel_Age']
psi_values = {}

def calculate_psi(expected, actual, buckets=10):
    def scale_range(data, buckets):
        return np.percentile(data, np.linspace(0, 100, buckets + 1))

    breakpoints = scale_range(expected, buckets)
    expected_percents = np.histogram(expected, bins=breakpoints)[0] / len(expected)
    actual_percents = np.histogram(actual, bins=breakpoints)[0] / len(actual)

    psi = np.sum((expected_percents - actual_percents) * np.log(expected_percents / actual_percents + 1e-6))
    return psi

for feature in features:
    psi = calculate_psi(train_slice[feature], current_slice[feature])
    psi_values[feature] = psi

psi_values

{'PolicyPremium': np.float64(0.15358451618886923),
 'Vessel_Value_USD': np.float64(0.10871097688003398),
 'Vessel_Age': np.float64(0.07347490115588416)}